---Black_Coffer--

In [2]:
# code to request multiple url 
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re

In [31]:
# read the url file into panda object 
df = pd.read_excel('Input.xlsx')


# loop for each row in df:
for index, row in df.iterrows():
    url = row['URL']
    url_index = row['URL_ID']

# request to the url 

    try:
        response = requests.get(url)
    except:
        print("can't get response of {}".format(url_index))

    #create beautifulsoul object
    try:
        soup = BeautifulSoup(response.content,'html.parser')
    except:
        print("can't get page of {}".format(url_index))
    
    #find title
    try:
        title = soup.find('h1').get_text()
    except:
        print("can't get title of {}".format(url_index))
        # continue
    #find article 
    article = " "
    try : 
        for p in soup.find_all('p'):
            article+= p.get_text()
    except:
        print("can't get text of {}".format(url_index))    
    
    # write title to the text file 
    file_name = 'TitleText/' +str(url_index) + '.txt'
    with open(file_name,'w') as file:
        file.write(title + '\n' + article)


can't get title of 44.0
can't get title of 57.0
can't get title of 144.0


In [4]:
# directories
text_dir = "TitleText/"
stopwords_dir = "StopWords/"
sentment_dir = "MasterDictionary/"


# load all stop words from stopword directory and  set in variable
stop_words = set()
for files in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir,files), 'r',encoding="latin-1") as f:
        stop_words.update(set(f.read().splitlines()))

#load all text files from the directory and store in the set variable 
docs=[]
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir,text_file),'r')as f:
        text =f.read()

        #tokenize given text 
        words = word_tokenize(text)

        #remove the stopwords from the tokens
        filtered_text = [word for word in words if word.lower() not in stop_words]

        #add each filtered token of each file into a list 
        docs.append(filtered_text)



        
        

In [6]:
#store positvie and negative words from the directory 
pos = set()
neg= set()

for files in os.listdir(sentment_dir):
    if files == 'positive-word.txt':
        with open(os.path.join((sentment_dir,files)),'r',encoding = 'latin-1') as f:
            pos.update(f.read().splitlines())
    else:
        with open(os.path.join(sentment_dir,files),'r',encoding='latin-1') as f:
            neg.update(f.read().splitlines())        
                        

# now collect positive and negative from each files:
positive_words=[]
negative_words=[]
positive_score=[]
negative_score=[]
polarity_score=[]
subjectively_score=[]

#calculate the score from the +ve and -ve 
# iterate through the list of docs 
for i in range(len(docs)):
    positive_words.append([word for word in docs[i] if word.lower() in pos ])
    negative_words.append([word for word in docs[i] if word.lower() in neg ])
    positive_score.append(len(positive_words[i]))
    negative_score.append(len(negative_words[i]))
    polarity_score.append((positive_score[i]-negative_score[i])/ ((positive_score[i] + negative_score[i]) + 0.000001))
    subjectively_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) +0.000001))



In [7]:
#average sentence length = the number of words / number of sentence 

# percentage of complex words = the number of complex words / number of words 

# fod index = 0.4 * (Average sentence Length + persentage of complex words )

avg_sentence_length = []
Percentage_of_complex_words = []
Fog_index = []
complex_word_count = []
avg_syllable_word_count = []

stop_words = set(stopwords.words('english'))
def measure (file):
    with open(os.path.join(text_dir,file),'r') as f:
        text = f.read()

        # remove punctuations
        text = re.sub(r'[^\w\s.]','',text)
        
        #split the given text file into sentences 
        sentence = text.split()

        #total number of sentence in a file 
        num_sentence = len(sentence)

        #total words in the file 
        words =  [word for word in text.split() if word.lower() not in stop_words]
        num_words = len(words)

        #complex words having syllable count is greater than 2
        #complex words are words in the text that contain more than two syllables
        complex_words= []
        for word in words:
            vowel = 'aeiou'
            syllable_count_word = sum(1 for letter in word if letter.lower() in vowel)
            if syllable_count_word>2:
                complex_words.append(word)

        #syllable count per word 
        #count the number of syllables in each word of the text by counting vowels present in each words.
        #exception handle  like words ending with "es" "ed"  not counting it .
        syllable_count = 0 
        syllable_words = []
        for word in words :
            if word.endswith('es'): 
                word = word[:-2]
            elif word.endswith('ed'):
                word = word[:-2]
            vowels = 'aeiou'
            syllable_count_word = sum (1 for letter in word if letter.lower() in vowels )
            if syllable_count_word >=1 :
                syllable_words.append(word)
                syllable_count += syllable_count_word
        
        avg_sentence_len = num_words / num_sentence
        avg_syllable_word_count = syllable_count / len(syllable_words)
        Percent_Complex_words = len(complex_words)/num_words
        Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

        return avg_sentence_len,Percent_Complex_words,Fog_Index,len(complex_words),avg_syllable_word_count
    
#iterate through each file or doc 
for file in os.listdir(text_dir):
    x,y,z,a,b = measure(file)
    avg_sentence_length.append(x)
    Percentage_of_complex_words.append(y)
    Fog_index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)
    


In [8]:
# word count and Average word length sum of the total number of characters in each word/total number of words
# we count the total cleaned words present in the text by
# removing the stopwords 
#removing the puntuation like ? ! , . from the word before counting 

def cleaned_words(file):
    with open(os.path.join(text_dir,file),'r') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]','', text) 
        words = [ word for word in text.split() if word.lower() not in stop_words]
        length = sum(len(word) for word in words)
        average_word_length = length/len(words)
    return len(words),average_word_length

word_count = [] 
average_word_length=[]
for file in os.listdir(text_dir):
    x ,y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)



In [9]:
# to calculate personal pronoun mentioned in the text , we use regex to find 
# the count of the words - "i," "we," , "my," "ours," and "us" . speacial care
# so that the country name US is not included in the list.

def count_personal_pronouns(file):
    with open (os.path.join(text_dir,file),'r') as f:
        text = f.read()
        personal_pronouns  = ["I", "we" , "my" , "ours" , "us"]
        count = 0
        for pronoun in personal_pronouns:
            count+= len(re.findall(r"\b" + pronoun + r"\b" , text)) # \b is used to match word boundaries
    return count
pp_count = []
for file in os.listdir(text_dir):
    x= count_personal_pronouns(file)
    pp_count.append(x)
    
print(pp_count)

[3, 3, 12, 5, 3, 6, 1, 3, 4, 38, 1, 1, 2, 8, 3, 3, 43, 8, 8, 2, 7, 35, 13, 8, 8, 3, 4, 7, 3, 2, 3, 1, 7, 1, 5, 8, 5, 4, 7, 1, 1, 4, 1, 2, 1, 3, 10, 2, 3, 1, 9, 2, 7, 3, 18, 13, 19, 8, 1, 3, 10, 2, 9, 7, 33, 12, 1, 16, 1, 1, 3, 1, 3, 4, 10, 5, 1, 5, 2, 1, 3, 14, 2, 1, 5, 6, 17, 19, 17, 1, 5, 9, 8, 12, 6, 1, 7, 14, 6, 1, 1, 6, 1, 11, 1, 3, 2, 6, 6, 4, 2, 4, 28, 13]


In [11]:
output_df = pd.read_excel("Output Data Structure.xlsx")

output_df.drop([44-37,57-37,144-37],axis=0, inplace=True)

variables = [positive_score,
             negative_score,
             polarity_score,
             subjectively_score,
             avg_sentence_length,
             Percentage_of_complex_words,
             Fog_index,
             avg_sentence_length,
             complex_word_count,
             word_count,
             avg_syllable_word_count,
             pp_count,
             average_word_length]
for i, var in enumerate(variables):
    output_df.iloc[:, i + 2] = var[:len(output_df)]
output_df.to_csv('Output_Data1.csv')